In [3]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversation.base import ConversationChain

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")
memory = ConversationBufferMemory()
chain = ConversationChain(
    llm=chat,
    memory=memory,
    verbose=True
)

chain.predict(input="olá")
# ou chain.run("olá")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: olá
AI:

> Finished chain.


'Olá! Como posso ajudá-lo hoje?'

In [4]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
Essa é uma conversa amigável entre um humano e uma IA

Conversa atual:
{history}
Human: {input}
AI:"""
)

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")
memory = ConversationBufferMemory()
chain = ConversationChain(
    prompt=prompt_template,
    llm=chat,
    memory=memory,
    verbose=True
)

chain.predict(input="Oi")



> Entering new ConversationChain chain...
Prompt after formatting:

Essa é uma conversa amigável entre um humano e uma IA

Conversa atual:

Human: Oi
AI:

> Finished chain.


'Olá! Como posso te ajudar hoje?'

### LLMChain

In [6]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains.llm import LLMChain

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

prompt = PromptTemplate.from_template(
    """
    Escolha o pior nome para mim sobre uma empresa que desenvolve soluções em {produto}
    """
)

chain = LLMChain(llm=chat, prompt=prompt)
produto = "LLMs com LangChain"
chain.run(produto)

'"LangChain Solutions Incríveis"'

### SimpleSequentialChain

In [7]:
from langchain.chains import SimpleSequentialChain

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

prompt = PromptTemplate.from_template(
    """
    Escolha o melhor nome para mim sobre uma empresa que desenvolve soluções em {produto}
    """
)
chain_nome = LLMChain(llm=chat, prompt=prompt)

prompt_descricao = PromptTemplate.from_template(
    """
    Sobre a empresa com nome {nome_empresa}.
    Informe uma descrição de até 20 palavras.
    """
)

chain_descricao = LLMChain(llm=chat, prompt=prompt_descricao)

chain = SimpleSequentialChain(
    chains=[chain_nome, chain_descricao],
    verbose=True
)

chain.run(input=produto)



> Entering new SimpleSequentialChain chain...
"LangSolve Solutions"
"LangSolve Solutions é uma empresa especializada em soluções linguísticas, oferecendo serviços de tradução e localização de alta qualidade."

> Finished chain.


'"LangSolve Solutions é uma empresa especializada em soluções linguísticas, oferecendo serviços de tradução e localização de alta qualidade."'

### SequentialChain

In [8]:
from langchain.chains import SequentialChain

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

prompt = PromptTemplate.from_template(
    """
    Qual um bom nome de empresa que desenvolve soluções em {produto}?
    """
)
chain_nome = LLMChain(llm=chat, prompt=prompt, output_key="nome_empresa")

prompt_descricao = PromptTemplate.from_template(
    """
    Sobre a empresa com o nome {nome_empresa} e que 
    produz seguinte {produto}.
    Informe uma descrição de até 30 palavras de atividades dessa empresa
    """
)
chain_descricao = LLMChain(llm=chat, prompt=prompt_descricao, 
                           output_key="descricao_empresa")

prompt_traducao = PromptTemplate.from_template(
    """
    Crie um nome em espanhol para a empresa de nome {nome_empresa}
    que possui a seguinte descrição {descricao_empresa}
    """
)
chain_traducao = LLMChain(llm=chat, prompt=prompt_traducao, output_key="nome_espanhol")

chain = SequentialChain(
    chains=[chain_nome, chain_descricao, chain_traducao],
    input_variables=["produto"],
    output_variables=["nome_empresa", "descricao_empresa", "nome_espanhol"],
    verbose=True
)

resposta = chain.invoke({"produto": produto})



> Entering new SequentialChain chain...

> Finished chain.


In [9]:
resposta["produto"]

'LLMs com LangChain'

In [10]:
resposta["nome_empresa"]

'LangChain Solutions'

In [11]:
resposta["descricao_empresa"]

'A LangChain Solutions produz soluções de aprendizado de máquina (LLM) com a tecnologia LangChain para diversos setores e aplicações.'

In [12]:
resposta["nome_espanhol"]

'Soluciones de Aprendizaje de Máquina LangChain'